# Phase 3 - Endpoints

In the last notebook, we combine and extend the previously developed rules to detect the interesting endpoints. We classify the extracted values in their context in order to decide which endpoint this might be. We do not call the rules from the other notebooks, but simply copied them since they are extend and modified in the context of the final tasks. In the first cell, we develop the rules using the gold standard as we directly evaluate the output.

In [45]:
%inputDir data-nlp
%outputDir ./temp/endpoint-out
%displayMode EVALUATION
%evalTypes ORR OSMean OSTime OSRate PFSMean PFSTime PFSRate
//%saveTypeSystem ./EndpointTypeSystem.xml
%saveTypeSystem ./../../../TypeSystem.xml

TYPESYSTEM TrialsTypeSystem;
TYPESYSTEM DKProCoreTypeSystem;


DECLARE EndpointInd, EndpointIndEnum;
DECLARE EndpointInd ORRInd, OSInd, PFSInd, OtherInd;
WORDLIST orrIndList = "orr_ind.txt";
WORDLIST osIndList = "os_ind.txt";
WORDLIST pfsIndList = "pfs_ind.txt";
WORDLIST otherIndList = "other_ind.txt";

MARKFAST(ORRInd, orrIndList, true);
MARKFAST(OSInd, osIndList, true);
MARKFAST(PFSInd, pfsIndList, true);
// it is sometimes easier to detect an entity correctly by also detecting 
// something else additionally.
MARKFAST(OtherInd, otherIndList, true);

// this kind of dictionary lookup can create overlapping indicators, remove the smaller ones.
EndpointInd->{ANY epi:EndpointInd{-> UNMARK(epi)};};

//median overall survival (OS)
e1:EndpointInd{-> e1.end=end.end} "(" e2:EndpointInd{-> UNMARK(e2)} end:")";
// enumeration fo indicator could influence the assignment later
((EndpointInd COMMA)* EndpointInd "and" POS_DET? @EndpointInd){-> EndpointIndEnum};

// hotfix sentences, broken char is a question mark
s1:Sentence{ENDSWITH(QUESTION)} s2:@Sentence{->UNMARK(s1),s2.begin=s1.begin};

DECLARE TimeInd (STRING kind);
DECLARE NumericValue (DOUBLE value, DOUBLE min, DOUBLE max, DOUBLE var);
DECLARE Unit (STRING kind);
DECLARE Value (NumericValue value, Unit unit);

TYPE RutaNUM = org.apache.uima.ruta.type.NUM;
DOUBLE num;
WORDTABLE NumberTable = "numbers.csv";
MARKTABLE(NumericValue, 2, NumberTable, true, 2, "", 2, "value" = 1);

BLOCK(NumericValues) Document{}{
    // normal numbers like 1,000.95
    ADDRETAINTYPE(WS);
    (RutaNUM{-PARTOF(NumericValue)} (COMMA RutaNUM{REGEXP("...")}) 
        (PERIOD RutaNUM)?){PARSE(num, "en")-> nv:NumericValue, nv.value=num};
    (RutaNUM{-PARTOF(NumericValue)} (PERIOD RutaNUM)?){PARSE(num, "en")-> nv:NumericValue, nv.value=num};
    (PERIOD{-PARTOF(NumericValue)} RutaNUM){PARSE(num, "en")-> nv:NumericValue, nv.value=num};

    // like twenty-two
    (nv1:NumericValue{PARTOF(W)-> UNMARK(nv1)} 
        SPECIAL.ct=="-" 
        nv2:NumericValue{PARTOF(W)-> UNMARK(nv2)}){-> nv:NumericValue, nv.value = (nv1.value+nv2.value)};
    // intervals like 39-54
    (nv1:NumericValue{-> UNMARK(nv1)} SPECIAL?
        SPECIAL.ct=="-" 
        nv2:@NumericValue{-> UNMARK(nv2)}){-> new:NumericValue, new.min=nv1.value, new.max=nv2.value};
    
    // NEW: we also need to detect variance like 3+/-0.4
    (nv1:@NumericValue{-> nv1.var=nv2.value,nv1.end=nv2.end} "+/-" nv2:NumericValue{-> UNMARK(nv2)});
    
    REMOVERETAINTYPE(WS);
}

// indicators for durations like months
WORDTABLE TimeIndTable = "time_ind.csv";
MARKTABLE(TimeInd, 1, TimeIndTable, "kind"=2);

// something that could hint an arm
DECLARE ArmInd;
// we should probably refactor this to a dictionary
(W{REGEXP("arm", true)} W{REGEXP("[abc]", true)} RutaNUM? COLON?){-> ArmInd};

// ignore text in brackets (within sentences only, not across)
// we reuse the rules of Chapter 2

DECLARE Open, Close, InBrackets;
DECLARE InBrackets InRoundBrackets, InSquareBrackets;
DECLARE Open RoundOpen, SquareOpen;
DECLARE Close RoundClose, SquareClose;

FOREACH(special) SPECIAL{}{
    special.ct=="("{->RoundOpen};
    special.ct==")"{->RoundClose};
    special.ct=="["{->SquareOpen};
    special.ct=="]"{->SquareClose};
}
ADDFILTERTYPE(InRoundBrackets);
FOREACH(open, false) RoundOpen{}{
    (open ANY[0,30]{-PARTOF(RoundClose)} RoundClose){-> InRoundBrackets};
}
REMOVEFILTERTYPE(InRoundBrackets);
ADDFILTERTYPE(InSquareBrackets);
FOREACH(open, false) SquareOpen{}{
    (open ANY[0,30]{-PARTOF(SquareClose)} SquareClose){-> InSquareBrackets};
}
REMOVEFILTERTYPE(InSquareBrackets);

// indicators that could be useful
DECLARE VSInd,CIInd;
// we could add a wordlist dictionary, but for new we simple classify the words
(W{REGEXP("v|vs|versus")} PERIOD?){-> VSInd};
// confidence interval indicator
(W{REGEXP("CI")}){-> CIInd};
    
// annotate the actual Value (also within brackets)
// 10%
(nv:NumericValue SPECIAL.ct=="%"{-> u:Unit,u.kind="percent"}){-> v:Value, v.value=nv, v.unit=u};
// 12 months
(nv:NumericValue SPECIAL.ct=="-"? ti:TimeInd{-> u:Unit,u.kind=ti.kind}){-> v:Value, v.value=nv, v.unit=u};

ADDFILTERTYPE(InBrackets);

// again ignoring brackets
// 10 (...) months
(nv:NumericValue{-PARTOF(Value)} SPECIAL.ct=="%"{-> u:Unit,u.kind="percent"}){-> v:Value, v.value=nv, v.unit=u};
(nv:NumericValue{-PARTOF(Value)} SPECIAL.ct=="-"? ti:TimeInd{-> u:Unit,u.kind=ti.kind}){-> v:Value, v.value=nv, v.unit=u};


// chunks that could be an arm indicator
Value (POS_ADP{-REGEXP("in")} W[1,2]{-PARTOF(TimeInd),-PARTOF(POS_CONJ),-PARTOF(NumericValue)}){-> ArmInd};
(POS_ADP{-REGEXP("in")} W[1,2]{-PARTOF(TimeInd),-PARTOF(POS_CONJ),-PARTOF(NumericValue)}){-> ArmInd} POS_CONJ @Value;

// now some additional logic for combined mentions
DECLARE ValueEnum;

// 25 vs. 8%
(nv1:NumericValue{-PARTOF(Value)-> v:Value, v.value=nv1, v.unit=v2.unit}
    VSInd v2:Value){-> ValueEnum};
// 2, 3, and 4 months
((NumericValue{-PARTOF(Value) -> v:Value, Value.value=NumericValue, Value.unit=v2.unit} COMMA?)+ 
    W{REGEXP("and")} v2:@Value){->ValueEnum};
// 2- and 3 months
((nv1:NumericValue{-PARTOF(Value)} SPECIAL.ct=="-"?){-> v:Value, v.value=nv1, v.unit=v2.unit}
    W{REGEXP("and")} v2:Value){->ValueEnum};

// no unit? like "was 0.89"
W{REGEXP("was")} nv:@NumericValue{-PARTOF(Value), nv.value > 0, nv.value < 1 
    -> u:Unit, u.kind="percent", v:Value, v.value=nv, v.unit=u};

((Value COMMA)* Value COMMA? "and" @Value{-PARTOF(ValueEnum)}){-> ValueEnum};

// even more distant combinations
ADDFILTERTYPE(ArmInd,COMMA,POS_CONJ);
v:Value nv:NumericValue{-PARTOF(Value)-> new:Value, new.value=nv, new.unit=v.unit};
nv:NumericValue{-PARTOF(Value)-> new:Value, new.value=nv, new.unit=v.unit} v:@Value ;

// some clean up of false positives
DECLARE NoValueContextInd;
W{REGEXP("patients?", true)->NoValueContextInd};
"confidence interval" -> NoValueContextInd;
v:Value{-> UNMARK(v)} NoValueContextInd;

// reset filtering
REMOVEFILTERTYPE(InBrackets,ArmInd,COMMA,POS_CONJ);

v:Value{-> UNMARK(v)} CIInd;
CIInd PM? v:Value{-> UNMARK(v)} SPECIAL;

// now the endpoints

// just two helper types for easier rules
DECLARE Percentage, Duration;
v:Value{v.unit.kind=="percent" -> Percentage};
v:Value{v.unit.kind!="percent" -> Duration};

DECLARE InCIBracket;
// brackets that define some confidence interval
InBrackets{CONTAINS(CIInd)-> InCIBracket};

DECLARE Ignored;
W{REGEXP("was|is|were|of|at|with|a")->Ignored};
ADDFILTERTYPE(Ignored);


// if we write rules for each endpoint seperately and include all the sequential patterns,
// then this won't end well. Too many unclear rules. Thus, we separate the sequential patterns
// from the semantics by introducting an additional construct "Endpoint", a relation combining
// potentially optional information (Values) with the indicator.
DECLARE Endpoint (EndpointInd indicator, Value mean, Value time, Value rate);

// a macro action for reducing feature assignments later on, for smaller rules
ACTION EP(ANNOTATION aInd, ANNOTATION aMean, ANNOTATION aTime, ANNOTATION aRate) 
    = CREATE(Endpoint, "indicator" = aInd, "mean" = aMean, "time" = aTime, "rate" = aRate);
ACTION Mean(ANNOTATION aInd, ANNOTATION aMean) 
    = CREATE(Endpoint, "indicator" = aInd, "mean" = aMean);
ACTION Time(ANNOTATION aInd, ANNOTATION aTime) 
    = CREATE(Endpoint, "indicator" = aInd, "time" = aTime);
ACTION Rate(ANNOTATION aInd, ANNOTATION aRate) 
    = CREATE(Endpoint, "indicator" = aInd, "rate" = aRate);

// we define different stage of sequential pattern form more specific to more general/simplier exmaples

// It is sometimes really helpful for the maintainability of the rules to add a representative 
// example as a comment where/why the rule should be applied.


// combinations with enums can get complicated
// we start with the simple rules, enums of 2 or 3 should be enough

//The 1-year PFS and OS rates were 93% and 100%
d:@Value{-PARTOF(Endpoint),PARTOF(Duration)}
    (EndpointIndEnum{CONTAINS(EndpointInd,2,2)} ValueEnum{-CONTAINS(Endpoint)}) ->{
    e1:EndpointInd{->Time(e1,d)} # e2:EndpointInd{->Time(e2,d)} # 
        v1:@Value{-PARTOF(Endpoint),PARTOF(Percentage)->Rate(e1,v1)} #
        v2:Value{-PARTOF(Endpoint),PARTOF(Percentage)->Rate(e2,v2)};
};


// Median PFS and median OS were 3.1 and 13.8 months
(EndpointIndEnum{CONTAINS(EndpointInd,3,3)} ValueEnum{-CONTAINS(Endpoint)}) ->{
    e1:EndpointInd # e2:EndpointInd # e3:EndpointInd # 
        v1:@Value{-PARTOF(Endpoint)->Mean(e1,v1)} #
        v2:Value{-PARTOF(Endpoint)->Mean(e2,v2)} #
        v3:Value{-PARTOF(Endpoint)->Mean(e3,v3)};
};
(EndpointIndEnum{CONTAINS(EndpointInd,2,2)} ValueEnum{-CONTAINS(Endpoint)}) ->{
    e1:EndpointInd # e2:EndpointInd # 
        v1:@Value{-PARTOF(Endpoint)->Mean(e1,v1)} #
        v2:Value{-PARTOF(Endpoint)->Mean(e2,v2)};
};

//the 1-year, 3-year, and 5-year survival rate was 89.2%, 50.9% and 27.5%
ValueEnum->{d:Value{PARTOF(Duration),-PARTOF(Endpoint)-> Time(i,d)};}
    i:@EndpointInd{-PARTOF(ORRInd)}
    ValueEnum->{p:Value{PARTOF(Percentage),-PARTOF(Endpoint)-> Rate(i,p)};};
//The 2-year post-ASCT OS (67% PMLCL vs. 53%, p = 0.78)
d:Value{-> Time(i,d)}
    ANY[0,3]{-PARTOF(Value),-PARTOF(EndpointInd)}
    i:@EndpointInd{-PARTOF(ORRInd)}
    InBrackets{-PARTOF(InCIBracket)}->{p:Value{PARTOF(Percentage),-PARTOF(Endpoint)-> Rate(i,p)};};
//The one-year survival rate was 55.8%
d:Value{PARTOF(Duration),-PARTOF(Endpoint)-> Time(i,d)} 
    i:@EndpointInd{-PARTOF(ORRInd)}
    p:Value{PARTOF(Percentage),-PARTOF(Endpoint)-> Rate(i,p)};
//a 57.6% overall survival (OS) at 62 months
p:Value{PARTOF(Percentage),-PARTOF(Endpoint)-> Rate(i,p)}
    i:@EndpointInd{-PARTOF(ORRInd)}
    d:Value{PARTOF(Duration),-PARTOF(Endpoint)-> Time(i,d)};
//a progression free survival (PFS) of 42% at 74 months
i:@EndpointInd {-PARTOF(ORRInd)}
    ANY[0,2]{-PARTOF(Value),-PARTOF(EndpointInd)}
    p:Value{PARTOF(Percentage),-PARTOF(Endpoint)-> Rate(i,p)}
    ANY[0,2]{-PARTOF(Value),-PARTOF(EndpointInd)}
    d:Value{PARTOF(Duration),-PARTOF(Endpoint)-> Time(i,d)};
//a progression free survival (PFS) at 74 months of 42% 
i:@EndpointInd{-PARTOF(ORRInd)}
    ANY[0,2]{-PARTOF(Value),-PARTOF(EndpointInd)}
    d:Value{PARTOF(Duration),-PARTOF(Endpoint)-> Time(i,d)}
    ANY[0,2]{-PARTOF(Value),-PARTOF(EndpointInd)}
    p:Value{PARTOF(Percentage),-PARTOF(Endpoint)-> Rate(i,p)};
    
// ORR (30% vs. 40%)
i:EndpointInd{-PARTOF(EndpointIndEnum)} InBrackets{-PARTOF(InCIBracket)}->{v:Value{-PARTOF(Endpoint)-> Mean(i,v)};};
// ORR 33%
i:EndpointInd{-PARTOF(EndpointIndEnum)}
    v:Value{-PARTOF(Endpoint)-> Mean(i,v)};
// the 30% ORR
v:Value{-PARTOF(Endpoint)-> Mean(i,v)} i:@EndpointInd{-PARTOF(EndpointIndEnum)};
// ORR bla 33%
i:EndpointInd{-PARTOF(EndpointIndEnum)}
    ANY[0,2]{-PARTOF(Value),-PARTOF(EndpointInd)}
    v:Value{-PARTOF(Endpoint)-> Mean(i,v)};

// fallbacks within sentences
Sentence{CONTAINS(EndpointInd)}->{
    i:EndpointInd{-PARTOF(EndpointIndEnum)}
        ANY+{-PARTOF(Endpoint),-PARTOF(EndpointInd)} 
        v:@Value{-PARTOF(Endpoint),-PARTOF(InBrackets)-> Mean(i,v)};
    ep:Endpoint
        ANY+{-PARTOF(EndpointInd),-PARTOF(Endpoint)} 
        v:@Value{-PARTOF(Endpoint),-PARTOF(InBrackets)-> Mean(ep.indicator,v)};
};

// now we create the actual endpoint annotations based on the relation
FOREACH(ep) Endpoint{}{
    ep.indicator.type==ORRInd->{ep.mean{->ORR};};
    ep.indicator.type==OSInd-> {
        ep.mean{->OSMean};
        ep.time{->OSTime};
        ep.rate{->OSRate};
    };
    ep.indicator.type==PFSInd-> {
        ep.mean{->PFSMean};
        ep.time{->PFSTime};
        ep.rate{->PFSRate};
    };
}


Processed 100/100 files. (took 3s)


Document,Type,F1,Precision,Recall,TP,FP,FN
11956647.txt.xmi,All,1.000,1.000,1.000,12,0,0
,ORR,1.000,1.000,1.000,5,0,0
,OSMean,1.000,1.000,1.000,3,0,0
,OSRate,1.000,1.000,1.000,1,0,0
,OSTime,1.000,1.000,1.000,1,0,0
,PFSMean,1.000,1.000,1.000,2,0,0
,PFSRate,0.000,0.000,0.000,0,0,0
,PFSTime,0.000,0.000,0.000,0,0,0
14962257.txt.xmi,All,1.000,1.000,1.000,5,0,0
,ORR,1.000,1.000,1.000,1,0,0


In the next cell, we investigate the annotations in order to interpret the errors reported above.

In [46]:
%resetCas
%inputDir ./temp/endpoint-out
%outputDir ./temp/trash
%displayMode CSV
%csvConfig SentenceWithError

TYPESYSTEM TrialsTypeSystem;
TYPESYSTEM DKProCoreTypeSystem;
TYPESYSTEM EndpointTypeSystem;

DECLARE SentenceWithError;
Sentence{OR(CONTAINS(FalsePositive),CONTAINS(FalseNegative))-> SentenceWithError};

//COLOR(Value, "#D0FFF0");
COLOR(EndpointInd, "#FFFFC0");
COLOR(TruePositive, "lightgreen");
COLOR(FalsePositive, "lightblue");
COLOR(FalseNegative, "pink");


Processed 100/100 files. (took 2s)
85 rows created.


17099879.txt.xmi,"The 1-year PFS and OS rates were 93%and 100%, respectively; and the 2-year PFS and OS rates were 86% and 86%,respectively."
17963264.txt.xmi,These results did not differ from that in younger patients < or =75 years in the OPTIMOX1 study with PFS 9.0 months (P =.63) and OS 20.2 months (P = .57).
18604722.txt.xmi,"The overall response rate (ORR) tosalvage chemotherapy (25% vs. 48%, p = 0.01) and 2-year OS after diagnosis of RR disease (15% vs. 34%, p = 0.018) was inferior in PMLCL patients."
18604722.txt.xmi,"The 2-yearpost-ASCT OS (67% PMLCL vs. 53%, p = 0.78) and PFS (57% PMLCL vs. 36%, p = 0.64) were similar."
18720480.txt.xmi,"The medianprogression-free survival (PFS) was 5.3 months (95% confidence interval [CI],3.7-7.5 months)."
18815728.txt.xmi,"By intent-to-treatanalysis, ORR was 21.1% (95% CI, 8.7-43.7) and disease control rate was 52.6%(95% CI 31.5-72.8) with four PRs and six SDs."
18815728.txt.xmi,Median PFS was 2.6 months (95% CI 2.2-2.9) andmedian OS was 9.8 months (95% CI 5.3-14.4) after median F/U of 15.4 months.
18981463.txt.xmi,"RESULTS: Median PFS for the 20-mg arm was 94 days, with 4- and 6-month PFS rateestimates of 42% and 24%, respectively."
18981463.txt.xmi,"Median PFS for the 200-mg arm was 64days, with 4- and 6-month PFS rate estimates of 41% and 32%, respectively."
19533023.txt.xmi,"Low MGMT expression, compared withhigh MGMT expression, showed no significant difference in ORR (25 vs. 8%), medianPFS (14 vs. 5 months) or OS (21 vs. 15 months)."
19628292.txt.xmi,"Efficacy was assessed inpatients who completed at least 1 cycle of treatment; ORR was 31.7% in arm Aversus 25.6% in arm B and DCR was 70.7% versus 56.4%, respectively."


Finally, we display and store the extracted information about the endpoints.

In [3]:
%resetCas
%inputDir ./temp/endpoint-out
%outputDir ./temp/trash
%displayMode CSV
%csvConfig TrialsEntity

TYPESYSTEM TrialsTypeSystem;
TYPESYSTEM DKProCoreTypeSystem;
TYPESYSTEM EndpointTypeSystem;

Processed 100/100 files. (took 5s)
1002 rows created.


11956647.txt.xmi,50%
11956647.txt.xmi,30.0%
11956647.txt.xmi,15months
11956647.txt.xmi,one-year
11956647.txt.xmi,55.8%
11956647.txt.xmi,10 months
11956647.txt.xmi,30.0%
11956647.txt.xmi,15 month
11956647.txt.xmi,10 month
11956647.txt.xmi,39-54%
11956647.txt.xmi,33%
